# Convolution Neural Network

## Get Data

In [16]:
import numpy as np
import keras
import tensorflow as tf

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

In [18]:
from keras.datasets import fashion_mnist
from helpers import fashion_mnist_label

In [19]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [20]:
label = fashion_mnist_label()
label

{0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'}

In [21]:
batch_size = 128
num_classes = 10
epochs=2

In [22]:
# input image dimensions
img_rows, img_cols = 28, 28

In [23]:
K.image_data_format()

'channels_last'

In [24]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [25]:
x_train_normalize =  x_train / 255
x_test_normalize = x_test / 255

In [26]:
# convert class vectors to binary class matrices
y_train_class = keras.utils.to_categorical(y_train, num_classes)
y_test_class = keras.utils.to_categorical(y_test, num_classes)

## Model 1: Simple Convolution

In [28]:
model_simple_conv = Sequential()
model_simple_conv.add(Conv2D(1, (3, 3), activation ="relu", input_shape=(28, 28, 1)))
model_simple_conv.add(Flatten())
model_simple_conv.add(Dense(100, activation='relu'))
model_simple_conv.add(Dense(10, activation='softmax'))

In [29]:
model_simple_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 1)         10        
_________________________________________________________________
flatten_1 (Flatten)          (None, 676)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               67700     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 68,720
Trainable params: 68,720
Non-trainable params: 0
_________________________________________________________________


In [30]:
model_simple_conv.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [31]:
tbCallBack_conv = keras.callbacks.TensorBoard(log_dir='logs/simple-conv')

In [32]:
%%time 

model_simple_conv.fit(x_train_normalize, y_train_class,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test_normalize, y_test_class),
                    callbacks=[tbCallBack_conv]
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 15s 244us/step - loss: 0.8056 - acc: 0.7219 - val_loss: 0.6278 - val_acc: 0.7737
Epoch 2/2
60000/60000 [==============================] - 14s 234us/step - loss: 0.5771 - acc: 0.7967 - val_loss: 0.5663 - val_acc: 0.7977
CPU times: user 48.9 s, sys: 5.81 s, total: 54.7 s
Wall time: 29.2 s


In [33]:
score = model_simple_conv.evaluate(x_test_normalize, y_test_class, verbose=1)

10000/10000 [==============================] - 2s 153us/step


In [34]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5662596433162689
Test accuracy: 0.7977


## Model 2: Convulation + Max Pooling

In [35]:
model_pooling_conv = Sequential()
model_pooling_conv.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model_pooling_conv.add(MaxPooling2D(pool_size=(2, 2)))
model_pooling_conv.add(Flatten())
model_pooling_conv.add(Dense(128, activation='relu'))
model_pooling_conv.add(Dense(num_classes, activation='softmax'))

In [36]:
model_pooling_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


In [37]:
model_pooling_conv.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [38]:
tbCallBack_pooling = keras.callbacks.TensorBoard(log_dir='logs/pooling-conv')

In [39]:
%%time 

model_pooling_conv.fit(x_train_normalize, y_train_class,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test_normalize, y_test_class),
                    callbacks=[tbCallBack_pooling]
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 40s 671us/step - loss: 0.9686 - acc: 0.6939 - val_loss: 0.6629 - val_acc: 0.7626
Epoch 2/2
60000/60000 [==============================] - 40s 661us/step - loss: 0.5903 - acc: 0.7910 - val_loss: 0.5703 - val_acc: 0.7938
CPU times: user 2min 48s, sys: 21 s, total: 3min 9s
Wall time: 1min 21s


In [40]:
score = model_pooling_conv.evaluate(x_test_normalize, y_test_class, verbose=1)

10000/10000 [==============================] - 3s 298us/step


In [41]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.570345180273056
Test accuracy: 0.7938


## Model 3: Convulation + Max Pooling + Dropout

In [42]:
model_dropout_conv = Sequential()
model_dropout_conv.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model_dropout_conv.add(Conv2D(64, (3, 3), activation='relu'))
model_dropout_conv.add(MaxPooling2D(pool_size=(2, 2)))
model_dropout_conv.add(Dropout(0.25))
model_dropout_conv.add(Flatten())
model_dropout_conv.add(Dense(128, activation='relu'))
model_dropout_conv.add(Dropout(0.5))
model_dropout_conv.add(Dense(num_classes, activation='softmax'))

In [43]:
model_dropout_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [44]:
model_dropout_conv.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [45]:
tbCallBack_dropout = keras.callbacks.TensorBoard(log_dir='logs/dropout-conv')

In [46]:
%%time 

model_dropout_conv.fit(x_train_normalize, y_train_class,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test_normalize, y_test_class),
                    callbacks=[tbCallBack_dropout]
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 197s 3ms/step - loss: 1.1826 - acc: 0.5768 - val_loss: 0.6959 - val_acc: 0.7331
Epoch 2/2
60000/60000 [==============================] - 203s 3ms/step - loss: 0.7409 - acc: 0.7260 - val_loss: 0.5998 - val_acc: 0.7785
CPU times: user 15min 42s, sys: 2min 35s, total: 18min 18s
Wall time: 6min 40s


In [47]:
score = model_dropout_conv.evaluate(x_test_normalize, y_test_class, verbose=1)

10000/10000 [==============================] - 11s 1ms/step


In [48]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5998393838405609
Test accuracy: 0.7785
